In [20]:
%matplotlib inline
import os
import numpy
from ipywidgets import interact, fixed

from supertomo.ui import arguments                     # Command line arguments
from supertomo.io import image_data                    # Data structure
from supertomo.reconstruction import fusion            # Image registration functions
from supertomo.ui import show
from supertomo.utils import itkutils


In [21]:
fuse_args = ("hela_full_test.hdf5 --dir=/home/sami/Data/SuperTomo2/Import "  
             "--max-nof-iterations=2  --first-estimate=constant " 
             "--fusion-method=summative --scale=100 --blocks=1 --pad=0").split()
            
print fuse_args

options = arguments.get_fusion_script_options(fuse_args)




['hela_full_test.hdf5', '--dir=/home/sami/Data/SuperTomo2/Import', '--max-nof-iterations=2', '--first-estimate=constant', '--fusion-method=summative', '--scale=100', '--blocks=1', '--pad=0']


In [22]:
full_path = os.path.join(options.working_directory,
                         options.data_file)

if not os.path.isfile(full_path):
    raise AttributeError("No such file: %s" % full_path)
elif not full_path.endswith(".hdf5"):
    raise AttributeError("Not a HDF5 file")

data = image_data.ImageData(full_path)

In [23]:
task = fusion.MultiViewFusionRL(data, options)

The original image size is 198 1024 1024
The fusion will be run with 1 blocks per dimension
The internal image size is 198 1024 1024


In [ ]:
import cProfile

task.estimate = numpy.ones(task.image_size, dtype=numpy.float64)
cProfile.run('task.compute_estimate()')

In [35]:
import numpy
import itertools

from accelerate.cuda.fft import FFTPlan, fft_inplace, ifft_inplace
from numba import cuda, vectorize
from supertomo.utils import generic_utils as genutils

class MultiViewFusionRLCuda(fusion.MultiViewFusionRL):
    def __init__(self, data, options):
        fusion.MultiViewFusionRL.__init__(self, data, options)

        threadpergpublock = 32, 32, 8
        self.blockpergrid = self.__best_grid_size(
            tuple(reversed(self.block_size)), threadpergpublock)

        FFTPlan(shape=self.block_size, itype=numpy.complex64,
                         otype=numpy.complex64)

        print('kernel config: %s x %s' % (self.blockpergrid, threadpergpublock))

    def compute_estimate(self):
        if "multiplicative" in self.options.fusion_method:
            estimate_new = numpy.ones(self.image_size, dtype=numpy.float32)
        else:
            estimate_new = numpy.zeros(self.image_size, dtype=numpy.float32)

        #d_estimate_new = cuda.to_device(estimate_new)

        # Iterate over blocks
        block_nr = 1
        for x, y, z in itertools.product(xrange(0, self.image_size[0], self.block_size[0]),
                                         xrange(0, self.image_size[1], self.block_size[1]),
                                         xrange(0, self.image_size[2], self.block_size[2])):

            index = numpy.array((x, y, z), dtype=int)
            if self.options.block_pad > 0:
                h_estimate_block = self.__get_padded_block(index.copy())
            else:
                h_estimate_block = self.estimate[index[0]:index[0] + self.block_size[0],
                                 index[1]:index[1] + self.block_size[1],
                                 index[2]:index[2] + self.block_size[2]]

            print "The current block is %i" % block_nr
            block_nr += 1
            
            d_estimate_block = cuda.to_device(h_estimate_block.astype(numpy.complex64))
        
            # Iterate over views
            for view in xrange(self.n_views):
                
                h_psf = genutils.expand_to_shape(self.psfs[view], d_estimate_block.shape)
                d_psf = cuda.to_device(h_psf.astype(numpy.complex64))
                
                h_adj_psf = genutils.expand_to_shape(self.adj_psfs[view], d_estimate_block.shape)
                d_adj_psf = cuda.to_device(h_adj_psf.astype(numpy.complex64))

                # Execute: cache = convolve(PSF, estimate), non-normalized
                fft_inplace(d_estimate_block)
                fft_inplace(d_psf)
                self.vmult(d_estimate_block, d_psf, out=d_estimate_block)

                ifft_inplace(d_estimate_block)

                h_estimate_block = d_estimate_block.copy_to_host().real

                # Execute: cache = data/cache
                self.data.set_active_image(view, self.options.channel, self.options.scale, "registered")
                h_data_block = self.data.get_registered_block(self.block_size, self.options.block_pad, index.copy())


                # ops_ext.inverse_division_inplace(cache, block)
                with numpy.errstate(divide="ignore"):
                    h_estimate_block = h_data_block / h_estimate_block
                    h_estimate_block[h_estimate_block == numpy.inf] = 0.0
                    h_estimate_block = numpy.nan_to_num(h_estimate_block)

                # Execute: cache = convolve(PSF(-), cache), inverse of non-normalized
                # Convolution with virtual PSFs is performed here as well, if
                # necessary

                d_estimate_block = cuda.to_device(h_estimate_block.astype(numpy.complex64))

                fft_inplace(d_estimate_block)
                fft_inplace(d_adj_psf)
                self.vmult(d_estimate_block, d_psf, out=d_estimate_block)

                ifft_inplace(d_estimate_block)

                h_estimate_block = d_estimate_block.copy_to_host().real

                # Update the contribution from a single view to the new estimate
                if self.options.block_pad == 0:
                    if "multiplicative" in self.options.fusion_method:
                        estimate_new[index[0]:index[0] + self.block_size[0],
                                     index[1]:index[1] + self.block_size[1],
                                     index[2]:index[2] + self.block_size[2]] *= h_estimate_block
                    else:

                        estimate_new[index[0]:index[0] + self.block_size[0],
                                     index[1]:index[1] + self.block_size[1],
                                     index[2]:index[2] + self.block_size[2]] += h_estimate_block
                else:
                    pad = self.options.block_pad

                    if "multiplicative" in self.options.fusion_method:
                        estimate_new[index[0]:index[0] + self.block_size[0],
                                     index[1]:index[1] + self.block_size[1],
                                     index[2]:index[2] + self.block_size[2]] *= h_estimate_block[pad:-pad]
                    else:
                        # print "The block size is ", self.block_size
                        estimate_new[index[0]:index[0] + self.block_size[0],
                                     index[1]:index[1] + self.block_size[1],
                                     index[2]:index[2] + self.block_size[2]] += h_estimate_block[pad:pad + self.block_size[0],
                                                                                                 pad:pad + self.block_size[1],
                                                                                                 pad:pad + self.block_size[2]]

    @staticmethod
    def __best_grid_size(size, tpb):
        bpg = numpy.ceil(numpy.array(size, dtype=numpy.float) / tpb).astype(numpy.int).tolist()
        return tuple(bpg)

    @staticmethod
    @vectorize(['complex64(complex64, complex64)'], target='cuda')
    def vmult(a, b):
        return a * b





In [36]:
task2 = MultiViewFusionRLCuda(data, options)

The original image size is 198 1024 1024
The fusion will be run with 1 blocks per dimension
The internal image size is 198 1024 1024
kernel config: (32, 32, 25) x (32, 32, 8)


In [37]:
task2.estimate = numpy.ones(task.image_size, dtype=numpy.float64)
cProfile.run('task2.compute_estimate()')

The current block is 1
         47299 function calls (44350 primitive calls) in 10.308 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.373    1.373   10.288   10.288 <ipython-input-35-e4a19321e8eb>:21(compute_estimate)
        1    0.020    0.020   10.309   10.309 <string>:1(<module>)
       18    0.000    0.000    0.000    0.000 <string>:8(__new__)
        4    0.000    0.000    0.000    0.000 UserDict.py:103(__contains__)
        1    0.000    0.000    0.000    0.000 UserDict.py:57(items)
        4    0.000    0.000    0.000    0.000 UserDict.py:91(get)
        3    0.000    0.000    0.000    0.000 __init__.py:248(__init__)
        3    0.000    0.000    0.000    0.000 __init__.py:277(name)
        3    0.000    0.000    0.000    0.000 __init__.py:327(__call__)
       42    0.000    0.000    0.000    0.000 __init__.py:389(__getitem__)
        1    0.000    0.000    0.000    0.000 __init__.py:485(__init__)
 

In [ ]:
task2.psfs[0].shape